In [1]:
from plotting_utils import plot_movement_trajectory_with_layout
import os

heartbeat_dir = os.path.join('EK_P02', 'Electrophysiology', 'VR', 'BBT-BIO-AAB023-2024-06-18_12-53-55')
heartbeat_path = os.path.join(heartbeat_dir, 'ExG [1].csv')
utc_path = os.path.join(heartbeat_dir, 'UTC.csv')

movement_dir = os.path.join('EK_P02', 'VR', 'pilot final', 'S001', 'trackers')
movement_path = os.path.join(movement_dir, 'camera_movement_T011.csv')


In [2]:
from calculations import calculate_total_rotation, df_diff
import pandas as pd
import numpy as np
from geometry_utils import get_ellipsoid_axes, get_focus, get_eccentricity, get_focus_and_eccentricity

movement_data = pd.read_csv(tracker_file).drop("timestamp", axis=1)
diff_frame = df_diff(movement_data)
diff_frame["dist"] = np.sqrt(diff_frame["pos_x"].pow(2) + diff_frame["pos_y"].pow(2) + diff_frame["pos_z"].pow(2))
diff_frame["speed"] = diff_frame["dist"] / diff_frame["time"]
print(diff_frame.mean())
import matplotlib.pyplot as plt
y = np.sort(diff_frame["speed"])
x = 100*np.arange(len(y))/len(y)

print(diff_frame[diff_frame["speed"] > 0.2].mean())
plt.plot(x,y)

points = np.transpose(np.stack((movement_data['pos_x'], movement_data['pos_z'])), axes=[1,0])
print(points.shape)

#print(get_focus_and_eccentricity(points))


NameError: name 'tracker_file' is not defined

In [4]:
import numpy as np
from geometry_utils import mean_edge_distance, mean_center_distance

center_point = np.array((-7,5))
corner_points = np.array([(-4,2),(-9,2),(-9,9),(-4,9)])

print(mean_center_distance(points, center_point))
print(mean_edge_distance(points, corner_points))

NameError: name 'points' is not defined

In [5]:
print(movement_data)

           time     pos_x     pos_y     pos_z      rot_x     rot_y  \
0      481.4744 -6.450725  1.610676  5.976768   18.69923  191.0628   
1      481.5030 -6.449114  1.612531  5.977763   17.93752  190.3711   
2      481.5665 -6.445739  1.616737  5.979550   16.17531  188.8127   
3      481.5835 -6.437787  1.626127  5.982200   10.59442  184.2375   
4      481.6032 -6.435087  1.628952  5.983236    8.90156  182.7538   
...         ...       ...       ...       ...        ...       ...   
15522  669.9332 -6.410432  1.643844  7.399132  344.49090  247.2068   
15523  669.9443 -6.410322  1.643813  7.399046  344.47230  247.1951   
15524  669.9551 -6.410256  1.643723  7.398874  344.46620  247.1842   
15525  669.9661 -6.410257  1.643744  7.398837  344.46520  247.1560   
15526  669.9771 -6.410212  1.643714  7.398784  344.44630  247.1548   

            rot_z            timestamp  
0      356.788400  06/18/2024 11:04:13  
1      357.026700  06/18/2024 11:04:13  
2      357.691100  06/18/2024 11:04:

In [105]:
from biosppy.signals.tools import filter_signal
import pandas as pd
import biosppy.signals.ecg as ecg
import numpy as np


def get_segment(data, start_time, end_time, timestamps):
    segment_points = data[np.logical_and(timestamps >= start_time, 
                                         timestamps < end_time)]
    return segment_points
    
def get_segment_mean_center_dist(segment, center_point):
    # transform points into np array of dimension [N, 2]
    points = np.transpose(np.stack((segment['pos_x'], segment['pos_z'])), axes=[1,0])
    
    center_dist = mean_center_distance(points, center_point)
    return center_dist

def get_segment_mean_edge_dist(segment, corner_points):
    points = np.transpose(np.stack((segment['pos_x'], segment['pos_z'])), axes=[1,0])
    return mean_edge_distance(points, corner_points)

def get_segment_heart_rate(segment, sampling_rate=256):
    r_peaks = find_r_peaks(segment, sampling_rate)
    rr_intervals = np.diff(r_peaks) / sampling_rate
    bpm = 60 / np.mean(rr_intervals)
    return bpm

def preprocess_ecg(signal, sampling_rate):
    #denoised_signal = cwt_denoise(signal)
    # Preprocess the ECG signal
    # Bandpass filter the ECG signal (0.5-40 Hz)
    filtered_signal = filter_signal(signal,
    #                                denoised_signal,
                                    ftype='FIR',
                                    band='bandpass',
                                    order=int(0.3 * sampling_rate),
                                    frequency=[0.1, 40],
                                    sampling_rate=sampling_rate) # check if they are the same and hf is getting filtered
    filtered_signal = filtered_signal[0]
    #filtered_signal = filtered_signal.reshape(-1)
    return filtered_signal

def find_r_peaks(signal, sampling_rate):
    # Find R peaks using the Pan-Tompkins algorithm
    r_peaks = ecg.engzee_segmenter(signal, sampling_rate=sampling_rate)[0]
    return r_peaks

def filter_rr_intervals(rr_intervals):
    # Remove RR intervals that are physiologically impossible
    # For example, intervals < 300ms or > 2000ms
    return rr_intervals[(rr_intervals >= 0.3) & (rr_intervals <= 2.0)]
'''
def calculate_rmssd(r_peaks, sampling_rate):
    # Calculate the RR intervals
    rr_intervals = np.diff(r_peaks) / sampling_rate
    
    # Calculate the successive differences of RR intervals
    rr_diff = np.diff(rr_intervals)
    
    # Calculate RMSSD
    rmssd = np.sqrt(np.mean(rr_diff ** 2))
    return rmssd
'''
def calculate_rmssd(r_peaks, sampling_rate):
    # Calculate the RR intervals in milliseconds
    rr_intervals_ms = np.diff(r_peaks) / sampling_rate * 1000
    
    # Calculate the successive differences of RR intervals
    rr_diff_ms = np.diff(rr_intervals_ms)
    
    # Square each value in rr_diff_ms
    rr_diff_squared = rr_diff_ms ** 2
    
    # Calculate the average of rr_diff_squared
    rr_diff_mean = np.mean(rr_diff_squared)
    
    # Calculate the square root of rr_diff_mean to get RMSSD
    rmssd = np.sqrt(rr_diff_mean)
    
    return rmssd

def get_segment_rmssd(signal_segment, sampling_rate=256):
    r_peaks = find_r_peaks(signal_segment, sampling_rate)
    rmssd = calculate_rmssd(r_peaks, sampling_rate)
    return rmssd

def get_segment_speed(movement_data):    
    diff_df = movement_data.drop('timestamp', axis=1).diff(axis=0)
    diff_df["dist"] = np.sqrt(diff_df["pos_x"].pow(2) + diff_df["pos_y"].pow(2) + diff_df["pos_z"].pow(2))
    speed = diff_df['dist']/diff_df['time']
    return np.mean(speed)

movement_data = pd.read_csv(movement_path)
heartrate_data = pd.read_csv(heartbeat_path)
utc_df = pd.read_csv(utc_path)
timestamp_diff = (utc_df['utc_timestamp']-utc_df['steady_timestamp']).astype(int).to_numpy()[0]

heartrate_timestamps = ((heartrate_data['steady_timestamp'] + timestamp_diff)/1e6).astype(int).to_numpy()

ecg_signal = heartrate_data['ExG [1]-ch1']
signal = preprocess_ecg(ecg_signal, 250)
r_peaks = find_r_peaks(signal, 250)
rmssd = calculate_rmssd(r_peaks, 250)


segment_length=10
step_size=1

# heart rate data 
#heart_rate_data

timestamps = pd.to_datetime(movement_data['timestamp']).apply(lambda x: x.timestamp()).astype(int).to_numpy()

start_time = np.floor(np.min(timestamps))

end_time = start_time + 90
segment_starts = np.arange(start_time, end_time-segment_length+1, step=step_size)
segment_ends = segment_starts + segment_length

segment_mean_center_dist = np.zeros(len(segment_starts))
segment_hr = np.zeros(len(segment_starts))
segment_mean_speed = np.zeros(len(segment_starts))
segment_rmssd = np.zeros(len(segment_starts))
segment_mean_edge_dist = np.zeros(len(segment_starts))

for idx, (start, end) in enumerate(zip(segment_starts, segment_ends)):
    movement_segment = get_segment(movement_data, start, end, timestamps)
    center_dist = get_segment_mean_center_dist(movement_segment, center_point)    
    segment_mean_center_dist[idx] = center_dist
    
    edge_dist = get_segment_mean_edge_dist(movement_segment, corner_points)
    segment_mean_edge_dist[idx] = edge_dist
    
    speed = get_segment_speed(movement_segment)
    segment_mean_speed[idx] = speed
    
    hr_segment = get_segment(signal, start+5, end+5, heartrate_timestamps)
    
    hr = get_segment_heart_rate(hr_segment)
    segment_hr[idx] = hr
    
    rmssd = get_segment_rmssd(hr_segment)
    segment_rmssd[idx] = rmssd
    

In [106]:
from scipy.stats import pearsonr

print(pearsonr(segment_mean_center_dist, segment_rmssd))

PearsonRResult(statistic=0.1744527880268711, pvalue=0.11931809573120168)
